In [34]:
import numpy as np
import pandas as pd

import yaml

from Bio.Emboss.Applications import WaterCommandline
from Bio import AlignIO
from Bio import Align
# http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec99

In [4]:
INPUT = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/data/exp9_MHC_IONTORRENT/mapping/KMA-1t1/output/mapping.frag.gz'

In [5]:
OLIGOS = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/tools/barcode_oligos/all_oligos.csv'

In [8]:
BRC = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/lib/barcode_design.yaml'

# Load

In [9]:
with open(BRC, 'r') as stream:
    brc = yaml.safe_load(stream)

In [10]:
brc

{'TSO': 'TTTCTTATATGGG',
 'R4': 'CGAGTACCATGGGCGTAAC',
 'N1': 'NNNNNN',
 'oligo_b': '25mer',
 'ANNEAL': 'CTACGCCTTTTGGGGAAGGTCACAC',
 'oligo_a': '25mer',
 'N2': 'NNNNNN',
 'P1': {'default': 'GACGCTGGCTGGAACTTC', 'A4000': 'GATGAAACCTGGGATCTTTTAC'}}

In [11]:
np.array(brc.values())

array(dict_values(['TTTCTTATATGGG', 'CGAGTACCATGGGCGTAAC', 'NNNNNN', '25mer', 'CTACGCCTTTTGGGGAAGGTCACAC', '25mer', 'NNNNNN', {'default': 'GACGCTGGCTGGAACTTC', 'A4000': 'GATGAAACCTGGGATCTTTTAC'}]),
      dtype=object)

In [12]:
oligos = pd.read_csv(OLIGOS)

/Users/herpov/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
oligos[oligos.oligo == 'B303']

,id_25mer,seq_25mer_fw,seq_25mer_rv,id_15mer_1,id_15mer_2,seq_15mer_fw,seq_15mer_rv,oligo
2307,bc25mer_2308,TAGGTATTCGGGCATTTCACTGTCA,TGACAGTGAAATGCCCGAATACCTA,NaN,NaN,NaN,NaN,B303


In [14]:
df = pd.read_csv(INPUT, sep='\t', header=None, names=['seq','hits','pos_x','pos_y','pos_z','barcode','read_id'])

In [15]:
df.barcode = df.barcode.str.split(' ', expand=True)[0]

In [16]:
df['gem'] = df.read_id.str.split('BX:Z:', expand=True)[1]

In [17]:
df[df.hits == 1].loc[1567,:]

seq        TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC...
hits                                                       1
pos_x                                                     47
pos_y                                                      0
pos_z                                                     87
barcode                                            A4000B303
read_id            O9A3T:00193:05223 BX:Z:GTCCTCATCTGTCCGT-1
gem                                       GTCCTCATCTGTCCGT-1
Name: 1567, dtype: object

# Tester

## Bio.Emboss.Applications WaterCommandline

In [21]:
BRC = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/brc_tmp.fa'
MAP = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/map_tmp.fa'
OUT = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/out_tmp.aln'

In [73]:
water = '/home/tuba-nobackup/shared/EMBOSS/bin/water'

In [76]:
cmd = WaterCommandline(water, asequence=BRC, bsequence=MAP, gapopen=10, gapextend=0.5, outfile=OUT)

In [77]:
cmd()

ApplicationError: Non-zero return code 127 from '/home/tuba-nobackup/shared/EMBOSS/bin/water -outfile=/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/water.aln -asequence=/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/brc_tmp.fa -bsequence=/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/map_tmp.fa -gapopen=10 -gapextend=0.5', message '/bin/sh: /home/tuba-nobackup/shared/EMBOSS/bin/water: No such file or directory'

In [40]:
OUT = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/experiments/exp13/run1/mhc/mapping/water.aln'

In [64]:
records = AlignIO.parse(OUT, "emboss")

In [65]:
for aln in records:
    print(aln[1].id)
    print(aln[0].seq)
    print(aln[1].seq)

7545
TTTCTTATAT-GGGCGAGTACCATGGGCGTAACNNNNNNGACACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCTCAGCGACCGACTANNNNNNGACGCTGGCTGGAACTTC
TTTCTTATATGGGGCGAGTACCATGGGCGTAACGTCGGGGATACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCACAGCGACCGACTATAAAGAGACGCTGGCTGGAACTTC
7780
TTTCTTATAT-GGGCGAGTACCATGGGCGTAACNNNNNNGACACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCTCAGCGACCGACTANNNNNNGACGCTGGCTGGAACTTC
TTTCTTATATGGGGCGATTACCATGGGCGTAACGTCGGGGATACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCTCAGCGACCGACTATAAAGAGACGCTGGCTGGAACTTC
1031
TTTCTTATAT-GGGCGAGTACCATGGGCGTAACNNNNNNGACACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCTCAGCGACCGACTANNNNNNGACGCTGGCTGGAACTTC
TTTCTTATATGGGGCNAGTANCATGGGCGTAACGTCGGGNATACCTAGCAGCGTGTTGTACTCCCTACGCCTTTTGGGGAAGGTCACACTGTTTCATAGCTCAGCGACCGACTATAAAGAGACGCTGGCTGGAACTTC


In [67]:
aln[1]

SeqRecord(seq=Seq('TTTCTTATATGGGGCNAGTANCATGGGCGTAACGTCGGGNATACCTAGCAGCGT...TTC', SingleLetterAlphabet()), id='1031', name='<unknown name>', description='1031', dbxrefs=[])

## Align.PairwiseAligner

In [35]:
aligner = Align.PairwiseAligner()

In [65]:
aligner.mode = 'local'
#aligner.mismatch_score = -10
aligner.open_gap_score = -1
aligner.extend_gap_score = -10

In [97]:
seq1 = "TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTCGGGCATTTCACTCGTCACTAACGCCTTTTTGGGGGAAGGCACACTGTTTCATAGCTCAGCGACCGACTACCTTTGACGCTGGCTGGAACTTCTGGAATTCTCGGGTGCAAGGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTACGCTGGCTGAACTTCTTGGAG"
seq2 = "CGAGTACCATGGGCGTAAC" #"TAGGTATTCGGGCATTTCACTGTCA" # #"TTTCTTATATGGG"
alignments = aligner.align(seq1, seq2)
alignment = alignments[0] # The first alignment is the best
alignment.aligned

(((13, 23), (23, 26), (26, 30)), ((0, 10), (11, 14), (15, 19)))

In [98]:
print(alignment)

TTTCTTATATGGGCGAGTACCAT-GGC-TAACACCGTTTAGGTACTTCGGGCATTTCACTCGTCACTAACGCCTTTTTGGGGGAAGGCACACTGTTTCATAGCTCAGCGACCGACTACCTTTGACGCTGGCTGGAACTTCTGGAATTCTCGGGTGCAAGGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTACGCTGGCTGAACTTCTTGGAG
             ||||||||||-|||-||||                                                                                                                                                                                              
             CGAGTACCATGGGCGTAAC                                                                                                                                                                                              



In [99]:
target = alignment.aligned[0]
target

((13, 23), (23, 26), (26, 30))

In [100]:
target[0][0]

13

In [102]:
target[-1][-1]

30

In [103]:
R4 = 30

In [104]:
BO = 36

In [105]:
seq1[R4:BO]

'ACCGTT'

In [45]:
for alignment in alignments:
    print(alignment)
    print(alignment.aligned)

TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
|||||||||||||                                 
TTTCTTATATGGG                                 

(((0, 13),), ((0, 13),))
TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
||||||||||||--|                               
TTTCTTATATGG--G                               

(((0, 12), (14, 15)), ((0, 12), (12, 13)))
TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
|||||||||||-|-|                               
TTTCTTATATG-G-G                               

(((0, 11), (12, 13), (14, 15)), ((0, 11), (11, 12), (12, 13)))
TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
||||||||||-||-|                               
TTTCTTATAT-GG-G                               

(((0, 10), (11, 13), (14, 15)), ((0, 10), (10, 12), (12, 13)))
TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
||||||||||||----|                             
TTTCTTATATGG----G                             

(((0, 12), (16, 17)), ((0, 12), (12, 13)))
TTTCTTATATGGGCGAGTACCATGGCTAACACCGTTTAGGTACTTC
||||||

In [38]:
alignment.aligned

(((0, 2), (4, 5)), ((0, 2), (2, 3)))